In [39]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import requests
import plotly.express as px

In [40]:
api_key = "ioCjwA1EbBvFdjMSq9JkwLYee9E8IXeh"

#https://site.financialmodelingprep.com/developer/docs/dashboard

# Grouping

In [81]:
sectors_forecast = {
    "bigtech": "^SP500-45",
    "staples": "^SP500-30",
    "Utilities": "^SP500-55",
    "finance": "^GSPC",
    "health": "^SP500-35",
    "discretionary": "^SP500-25",
    "real estate": "^SP500-60",
}

sectors_annual_average_return = {           #BASED ON HISTORY DATA
    "^SP500-45": 0.275590,
    "^SP500-30": 0.086390,
    "^SP500-55": 0.060968,
    "^GSPC": 0.165529,
    "^SP500-35": 0.114558,
    "^SP500-25": 0.215885,
    "^SP500-60": 0.074752,
}

inflation_rate = 0.04

t = 5


sectors_trailingPE = {           #ENTER MANUALLY
    "^SP500-45": 28.5,
    "^SP500-30": 21.8,
    "^SP500-55": 18.9,
    "^GSPC": 17.7,
    "^SP500-35": 18.0,
    "^SP500-25": 27.8,
    "^SP500-60": 39.7,
}

In [100]:
group_by_metrics = {
    "Market Price 5Y Change": "G",
    "EPS 5Y Change": "G",
    "YoY Growth": "G",
    "Multiplicative method": "G",
    "CAGR method": "G",
    
    "P/S": "V",
    "P/E": "V",
    "EV/EBITDA": "V",
    "PEG": "V",
    "ROI": "V",
    "ROA": "V",
    "ROE": "V",
    
    # "Discounted Model": "V",  # <-- использовано для прогноза по денежным потокам
    
    "DPR": "Q",
    "Cash Flow 5Y Change": "Q",
    "EBITDA 5Y Change": "Q",
    "Sales 5Y Change": "Q",
    "Net Income 5Y Change": "Q",
    "Net Debt/EBITDA": "Q",
    "Free Cash Flow": "Q",  # <-- дополнительная метрика
    "Dividend Yield": "Q"  # <-- дополнительная метрика
}

group_by_aspects = {
    "Market Price 5Y Change": "Historical Growth",
    "EPS 5Y Change": "Historical Growth",
    "YoY Growth": "Historical Growth",
    
    "Multiplicative method": "Future Growth Predictions",
    "CAGR method": "Future Growth Predictions",
    
    "P/S": "Market-Based Ratios",
    "P/E": "Market-Based Ratios",
    "EV/EBITDA": "Market-Based Ratios",
    "PEG": "Market-Based Ratios",
    
    "ROI": "Return-Based Ratios",
    "ROA": "Return-Based Ratios",
    "ROE": "Return-Based Ratios",
    
    # "Discounted Model": "V",  # <-- использовано для прогноза по денежным потокам
    
    "DPR": "Dividend Metrics",
    "Dividend Yield": "Dividend Metrics",  # <-- добавлено для дивидендов

    "Cash Flow 5Y Change": "Profitability & Stability",
    "Net Income 5Y Change": "Profitability & Stability",
    "EBITDA 5Y Change": "Profitability & Stability",
    
    "Sales 5Y Change": "Size & Growth Stability",
    
    "Net Debt/EBITDA": "Leverage",
    "FCF Yield": "Profitability & Stability"  # <-- добавлено для фри-кэш-флоу
}


aspects_metrics = {
    "Historical Growth": ["Market Price 5Y Change", "EPS 5Y Change", "YoY Growth"],
    "Future Growth Predictions": ["Multiplicative method", "CAGR method"],
    "Market-Based Ratios": ["P/S", "P/E", "EV/EBITDA", "PEG"],
    "Return-Based Ratios": ["ROI", "ROA", "ROE"],
    # "Valuation": ["Discounted Model"],
    "Dividend Metrics": ["DPR", "Dividend Yield"],
    "Profitability & Stability": ["Cash Flow 5Y Change", "Net Income 5Y Change", "EBITDA 5Y Change", "FCF Yield"],
    "Size & Growth Stability": ["Sales 5Y Change"],
    "Leverage": ["Net Debt/EBITDA"]
}


gqv_for_aspects = {
    "G": ["Historical Growth", "Future Growth Predictions"],
    # "V": ["Market-Based Ratios", "Valuation"],
    "V": ["Market-Based Ratios"],
    "Q": ["Dividend Metrics", "Profitability & Stability", "Size & Growth Stability", "Leverage"]
}

# PARTIALLY

## Forecasting price (Multiplicative method, CAGR method)

In [119]:
def multiplicative_method(stock_growth, industry_growth, beta, dividend_yield=0, t=1, inflation_rate=0):
  
    # Вычисляем факторы роста
    aim_growth_upper = ((1 + stock_growth) * ((1 + 0.01 * beta) * (1 + industry_growth)))
    aim_growth_lower = ((1 + stock_growth) * ((1 - 0.01 * beta) * (1 + industry_growth)))

    # Учитываем дивиденды, если применимо
    if dividend_yield > 0:
        inflation_adjustment = (1 - inflation_rate) ** t
        dividend_factor = dividend_yield * t * inflation_adjustment
        aim_growth_upper *= 1 + dividend_factor
        aim_growth_lower *= 1 + dividend_factor


    # Средняя цена
    avg_growth = (aim_growth_upper + aim_growth_lower) / 2

    return avg_growth, aim_growth_lower, aim_growth_upper



def cagr_revenue_model(revenue_current, cagr_last_5y, ev_current, beta, t=5):
    # Прогнозируем доход через t лет
    revenue_forecast = revenue_current * (1 + cagr_last_5y) ** t


    # Прогнозируем EV через t лет
    ev_forecast = revenue_forecast * (ev_current / revenue_current)

    # Прогнозируем CAGR через t лет
    cagr_forecast = (ev_forecast / ev_current) ** (1 / t) - 1

    # Вычисляем факторы роста через t лет
    aim_growth_upper = ((1 + cagr_forecast) ** t - 1) * (1 + 0.01 * beta)
    aim_growth_lower = ((1 + cagr_forecast) ** t - 1) * (1 - 0.01 * beta)

    # Средняя цена за t лет
    avg_growth = (aim_growth_upper + aim_growth_lower) / 2
    return avg_growth, aim_growth_lower, aim_growth_upper

## GET DATA

In [102]:
stock = yf.Ticker('JPM')
info = stock.info
print(info)




{'address1': '383 Madison Avenue', 'city': 'New York', 'state': 'NY', 'zip': '10179', 'country': 'United States', 'phone': '212 270 6000', 'website': 'https://www.jpmorganchase.com', 'industry': 'Banks - Diversified', 'industryKey': 'banks-diversified', 'industryDisp': 'Banks - Diversified', 'sector': 'Financial Services', 'sectorKey': 'financial-services', 'sectorDisp': 'Financial Services', 'longBusinessSummary': 'JPMorgan Chase & Co. operates as a financial services company worldwide. It operates through four segments: Consumer & Community Banking (CCB), Corporate & Investment Bank (CIB), Commercial Banking (CB), and Asset & Wealth Management (AWM). The CCB segment offers deposit, investment and lending products, cash management, and payments and services; mortgage origination and servicing activities; residential mortgages and home equity loans; and credit cards, auto loans, leases, and travel services to consumers and small businesses through bank branches, ATMs, and digital and t

In [104]:
multiplicative_forecasting = {}     #ticker: [average, upper, lower]
cagr_forecasting = {}     #ticker: [average, upper, lower]

def get_data(sector_ticker, tickers):

    data = []

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Получаем финансовые данные за последние 5 лет из FMP
        url_financials = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?limit=5&apikey={api_key}"
        url_cash_flow = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?limit=5&apikey={api_key}"
        url_profile = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={api_key}"
        
        financials = requests.get(url_financials).json()
        cash_flows = requests.get(url_cash_flow).json()
        profile = requests.get(url_profile).json()

        # Проверка на наличие данных
        ebitda_5y_change = None
        cash_flow_5y_change = None
        sales_5y_change = None
        net_income_5y_change = None
        market_cap_5y_change = None

        beta = info.get("beta")
        dividend_yield = info.get("dividendYield", 0)
        ev_current = float(info.get("enterpriseValue", 1))
        revenue_current = float(info.get("totalRevenue", 1))
        pe_company = info.get("trailingPE", 0)

        #Price Forecasting
        revenue_5y_ago = financials[4]["revenue"]
        # Проверка на наличие выручки за 5 лет назад
        if revenue_5y_ago > 0:
            cagr_last_5y = ((revenue_current / revenue_5y_ago) ** (1 / t)) - 1
        else:
            cagr_last_5y = 0


        sector_pe = sectors_trailingPE[sector_ticker]  # Можно интегрировать API для получения среднего значения P/E сектора
        # Проверка на существование данных
        if sector_pe is not None and sector_pe > 0:
            # Расчет разницы P/E
            pe_diff_percentage = ((pe_company - sector_pe) / sector_pe)
        else:
            pe_diff_percentage = None  # Если данных нет, оставляем None

        

        multiplicative_forecasting[ticker] = list(multiplicative_method(pe_diff_percentage, sectors_annual_average_return[sector_ticker], beta, dividend_yield, t, inflation_rate))
        cagr_forecasting[ticker] = list(cagr_revenue_model(revenue_current, cagr_last_5y, ev_current, beta, t))
        


        if len(financials) >= 2:
            # EBITDA 5Y Change
            ebitda_current = float(financials[0].get("ebitda", 0))
            ebitda_5y_ago = float(financials[-1].get("ebitda", 1))
            ebitda_5y_change = ((ebitda_current - ebitda_5y_ago) / ebitda_5y_ago) * 100

            # Sales 5Y Change
            sales_current = float(financials[0].get("revenue", 0))
            sales_5y_ago = float(financials[-1].get("revenue", 1))
            sales_5y_change = ((sales_current - sales_5y_ago) / sales_5y_ago) * 100

            # Net Income 5Y Change
            net_income_current = float(financials[0].get("netIncome", 0))
            net_income_5y_ago = float(financials[-1].get("netIncome", 1))
            net_income_5y_change = ((net_income_current - net_income_5y_ago) / net_income_5y_ago) * 100

        if len(cash_flows) >= 2:
            # Cash Flow 5Y Change
            cash_flow_current = float(cash_flows[0].get("operatingCashFlow", 0))
            cash_flow_5y_ago = float(cash_flows[-1].get("operatingCashFlow", 1))
            cash_flow_5y_change = ((cash_flow_current - cash_flow_5y_ago) / cash_flow_5y_ago) * 100

            #Free CashFlow
            free_cash_flow = cash_flows[0].get("freeCashFlow", 0)

        if profile:
            # Market Cap 5Y Change

            market_cap_current = profile[0].get("mktCap")
            market_cap_5y_ago = profile[0].get("mktCap5Y")
            if market_cap_current and market_cap_5y_ago:
                market_cap_5y_change = ((market_cap_current - market_cap_5y_ago) / market_cap_5y_ago) * 100


        FCF_yield = (free_cash_flow / market_cap_current)

        # Сбор данных для таблицы
        row = {
            
            "Ticker": ticker,
            #G
            "Market Price 5Y Change": stock.history(period="5y")["Close"].pct_change().sum(),

            "EPS 5Y Change": (info.get("forwardEps")/info.get("trailingEps", 0)) * 100,
            "YoY Growth": info.get("revenueGrowth"),

            "Multiplicative method": multiplicative_forecasting[ticker][0],
            "CAGR method": cagr_forecasting[ticker][0],

            #V
            
            "P/S": info.get("priceToSalesTrailing12Months"),
            "P/E": info.get("trailingPE"),
            "EV/EBITDA": info.get("enterpriseToEbitda"),
            "PEG": info.get("trailingPegRatio"),
            "ROI": info.get("returnOnInvestment"),
            "ROA": info.get("returnOnAssets"),
            "ROE": info.get("returnOnEquity"),
            
            #Q
            "DPR": info.get("payoutRatio"),
            "Dividend Yield": dividend_yield,
            "Cash Flow 5Y Change": cash_flow_5y_change,
            "EBITDA 5Y Change": ebitda_5y_change,
            "Sales 5Y Change": sales_5y_change,
            "Net Income 5Y Change": net_income_5y_change,
            "Net Debt/EBITDA": info.get("totalDebt") / info.get("ebitda", 1),
            "FCF Yield": FCF_yield
            
        }
        data.append(row)

    return pd.DataFrame(data)



## ANALYSIS

In [105]:
def analysis(df):

    # 1. Нахождение максимальных значений для каждой метрики
    max_values = df.drop(columns=["Ticker"]).max()

    # 2. Нормализуем метрики по максимальному значению
    for col in df.columns[1:]:  # Все столбцы кроме "Ticker"
        df[col] = df[col] / max_values[col]

    # 3. Создание подгрупп для вычисления средних
    def calculate_group_averages(group_columns):
        return df[group_columns].mean(axis=1)

    # Перебираем ключи из gqv_for_aspects и вычисляем средние
    for group, aspects in gqv_for_aspects.items():
        group_columns = []

        print(group)
        
        # Собираем колонки для каждой подгруппы
        for aspect in aspects:
            group_columns.extend(aspects_metrics[aspect])  # Сразу добавляем метрики из списка

        print(group_columns)

        # Вычисляем среднее для каждой группы
        df[f'{group}_avg'] = calculate_group_averages(group_columns)

    # 4. Вычисление среднего по меткам G, Q, V
    df['GQV_avg'] = df[['G_avg', 'V_avg', 'Q_avg']].mean(axis=1)


    return df

## VISUALIZATION

In [ ]:
def plot_3d(df):
    # 5. Строим 3D график с Plotly
    fig = px.scatter_3d(df, x='G_avg', y='V_avg', z='Q_avg',
                        color='Ticker', hover_data=['Ticker'],
                        title='3D Analysis of Metrics')

    fig.update_layout(scene=dict(
                        xaxis_title='Growth (G)',
                        yaxis_title='Valuation (V)',
                        zaxis_title='Quality (Q)'
                        ))

    fig.show()

In [114]:


def plot_3d(df):
    # Добавляем точки MINIMUM и MAXIMUM
    minimum_point = pd.DataFrame({'G_avg': [0], 'V_avg': [0], 'Q_avg': [0], 'Ticker': ['MINIMUM']})
    maximum_point = pd.DataFrame({'G_avg': [1], 'V_avg': [1], 'Q_avg': [1], 'Ticker': ['MAXIMUM']})

    # Добавляем их в DataFrame
    df = pd.concat([df, minimum_point, maximum_point], ignore_index=True)

    # Строим 3D график с Plotly
    fig = px.scatter_3d(df, x='G_avg', y='V_avg', z='Q_avg',
                        color='Ticker', hover_data=['Ticker'],
                        title='3D Analysis of Metrics')

    fig.update_layout(scene=dict(
                        xaxis_title='Growth (G)',
                        yaxis_title='Valuation (V)',
                        zaxis_title='Quality (Q)',
                        xaxis=dict(range=[0, 1]),  # Ограничиваем оси от 0 до 1
                        yaxis=dict(range=[0, 1]),
                        zaxis=dict(range=[0, 1])
                        ))

    fig.show()





 # 

### ------------------------------------------------- 

# 

In [116]:
tickers = ['MA', 'JPM']  
sector_ticker = "^GSPC"


df = get_data(sector_ticker=sector_ticker, tickers=tickers)
print("GET DATA COMPLETED")

GET DATA COMPLETED


In [117]:
df = analysis(df = df)
print("ANALUYSIS COMPLETED")


G
['Market Price 5Y Change', 'EPS 5Y Change', 'YoY Growth', 'Multiplicative method', 'CAGR method']
V
['P/S', 'P/E', 'EV/EBITDA', 'PEG']
Q
['DPR', 'Dividend Yield', 'Cash Flow 5Y Change', 'Net Income 5Y Change', 'EBITDA 5Y Change', 'FCF Yield', 'Sales 5Y Change', 'Net Debt/EBITDA']
ANALUYSIS COMPLETED


In [120]:
plot_3d(df = df)

0    NaN
1    NaN
Name: PEG, dtype: object


TypeError: list indices must be integers or slices, not str